In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import gc
import xgboost as xgb
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
max_num_features = 10
pad_size = 1
boundary_letter = -1
space_letter = 0
max_data_size = 320000

In [3]:
out_path = r'.'
df = pd.read_csv(r'input/en_train.csv')

In [4]:
x_data = []
y_data =  pd.factorize(df['class'])
labels = y_data[1]
y_data = y_data[0]
gc.collect()
for x in df['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    x_data.append(x_row)

In [5]:
def context_window_transform(data, pad_size):
    pre = np.zeros(max_num_features)
    pre = [pre for x in np.arange(pad_size)]
    data = pre + data + pre
    neo_data = []
    for i in np.arange(len(data) - pad_size * 2):
        row = []
        for x in data[i : i + pad_size * 2 + 1]:x_train = x_data
y_train = y_data
gc.collect()

x_train, x_valid, y_train, y_valid= train_test_split(x_train, y_train,
                                                      test_size=0.1, random_state=2017)
gc.collect()
            row.append([boundary_letter])
            row.append(x)
        row.append([boundary_letter])
        neo_data.append([int(x) for y in row for x in y])
    return neo_data

In [6]:
x_data = x_data[:max_data_size]
y_data = y_data[:max_data_size]
x_data = np.array(context_window_transform(x_data, pad_size))
gc.collect()
x_data = np.array(x_data)
y_data = np.array(y_data)

In [7]:
print('Total number of samples:', len(x_data))
print('Use: ', max_data_size)
#x_data = np.array(x_data)
#y_data = np.array(y_data)

print('x_data sample:')
print(x_data[0])
print('y_data sample:')
print(y_data[0])
print('labels:')
print(labels)


Total number of samples: 320000
Use:  320000
x_data sample:
[ -1   0   0   0   0   0   0   0   0   0   0  -1  66 114 105 108 108  97
 110 116  97 105  -1 105 115   0   0   0   0   0   0   0   0  -1]
y_data sample:
0
labels:
Index(['PLAIN', 'PUNCT', 'DATE', 'LETTERS', 'CARDINAL', 'VERBATIM', 'DECIMAL',
       'MEASURE', 'MONEY', 'ORDINAL', 'TIME', 'ELECTRONIC', 'DIGIT',
       'FRACTION', 'TELEPHONE', 'ADDRESS'],
      dtype='object')


In [23]:
x_train = x_data
y_train = y_data
gc.collect()

x_train, x_valid, y_train, y_valid= train_test_split(x_train, y_train,
                                                      test_size=0.1, random_state=2017)
gc.collect()
num_class = len(labels)
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)
watchlist = [(dvalid, 'valid'), (dtrain, 'train')]

param = {'objective':'multi:softmax',
         'eta':'0.3', 'max_depth':10,
         'silent':1, 'nthread':-1,
         'num_class':num_class,
         'eval_metric':'merror'}
model = xgb.train(param, dtrain, 50, watchlist, early_stopping_rounds=20,
                  verbose_eval=10)
gc.collect()

pred = model.predict(dvalid)
pred = [labels[int(x)] for x in pred]
y_valid = [labels[x] for x in y_valid]
x_valid = [ [ chr(x) for x in y[2 + max_num_features: 2 + max_num_features * 2]] for y in x_valid]
x_valid = [''.join(x) for x in x_valid]
x_valid = [re.sub('a+$', '', x) for x in x_valid]

gc.collect()

df_pred = pd.DataFrame(columns=['data', 'predict', 'target'])
df_pred['data'] = x_valid
df_pred['predict'] = pred
df_pred['target'] = y_valid
df_pred.to_csv(os.path.join(out_path, 'pred.csv'))

df_erros = df_pred.loc[df_pred['predict'] != df_pred['target']]
df_erros.to_csv(os.path.join(out_path, 'errors.csv'), index=False)

model.save_model(os.path.join(out_path, 'xgb_model'))


[22:14:09] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	valid-merror:0.00759	train-merror:0.00778
Multiple eval metrics have been passed: 'train-merror' will be used for early stopping.

Will train until train-merror hasn't improved in 20 rounds.
[10]	valid-merror:0.00547	train-merror:0.00354
[20]	valid-merror:0.00487	train-merror:0.00207
[30]	valid-merror:0.00478	train-merror:0.00099
[40]	valid-merror:0.00475	train-merror:0.00050
[49]	valid-merror:0.00478	train-merror:0.00026


In [27]:
df1 = pd.read_csv(r'errors.csv')

In [28]:
df1

,data,predict,target
0,ECCLESIAST,PLAIN,LETTERS
1,2,CARDINAL,DIGIT
2,NORML,LETTERS,PLAIN
3,-,PUNCT,PLAIN
4,ISA,LETTERS,PLAIN
...,...,...,...
148,ec,PLAIN,LETTERS
149,gibt,PLAIN,LETTERS
150,Esaal,PLAIN,LETTERS
151,March 1,PLAIN,DATE


In [31]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(pred, y_valid)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.52%
